In [ ]:
class XWords(object):
    def __init__(self, entries_file, geometries_file, blockers_file):
        self.entries_file    = entries_file
        self.geometries_file = geometries_file
        self.blockers_file   = blockers_file
        with open(self.entries_file) as f:
            self.entries    = eval(f.read())
        with open(self.geometries_file) as f:
            self.geometries = eval(f.read())
        with open(self.blockers_file) as f:
            self.blockers   = eval(f.read())

import os
_dir_    = '../../../data/crossword_puzzle_screenshots/'
_files_  = os.listdir(_dir_)
entries_file    = None
geometries_file = None
blockers_file   = None
for _file_ in _files_:
    if   'entries'    in _file_: entries_file    = _dir_ + _file_
    elif 'geometries' in _file_: geometries_file = _dir_ + _file_
    elif 'blockers'   in _file_: blockers_file   = _dir_ + _file_
xwords = XWords(entries_file, geometries_file, blockers_file)